In [1]:
# Pfad zu Embedding festlegen

#extract = "origin"
extract = "geschichtsbuch"

json_embedding_geschichtsbuch = "docs/embeddings_geschichtsbuch.json"
json_embedding_origin = "docs/embeddings_dan_brown.json"

chroma_collection_geschichtsbuch = "geschichtsbuch_collection"
chroma_collection_origin = "dan_brown_collection"

chroma_db_geschichtsbuch = "docs/geschichtsbuch_chroma_db"
chroma_db_origin = "docs/dan_brown_chroma_db"


if extract == "geschichtsbuch": 
    json_embedding_path = json_embedding_geschichtsbuch
    chroma_collection = chroma_collection_geschichtsbuch
    chroma_collection_db = chroma_db_geschichtsbuch
else:
    json_embedding_path = json_embedding_origin
    chroma_collection = chroma_collection_origin
    chroma_collection_db = chroma_db_origin

print(f"json_embedding_path is: {json_embedding_path}")
print(f"chroma_collection is: {chroma_collection}")
print(f"chroma_collection_db is: {chroma_collection_db}")

json_embedding_path is: docs/embeddings_geschichtsbuch.json
chroma_collection is: geschichtsbuch_collection
chroma_collection_db is: docs/geschichtsbuch_chroma_db


# LLM Q&A System mit GPT-4o

Frage-Antwort-System mit ChromaDB und OpenAI GPT-4o

## Libraries importieren

In [4]:
!pip install openai

  Using cached openai-2.15.0-py3-none-any.whl.metadata (29 kB)
  Using cached jiter-0.12.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached openai-2.15.0-py3-none-any.whl (1.1 MB)
Using cached jiter-0.12.0-cp311-cp311-macosx_11_0_arm64.whl (320 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [openai]2m2/3 [openai]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [5]:
import chromadb
import torch
from transformers import AutoTokenizer, AutoModel
from openai import OpenAI
import json

print("Libraries importiert")

Libraries importiert


## OpenAI API Key laden

In [6]:
# OpenAI API Key aus Datei laden
with open('open_ai_key.txt', 'r') as f:
    api_key = f.read().strip()

# OpenAI Client initialisieren
client = OpenAI(api_key=api_key)

print("OpenAI API Key geladen")
print(f"  Key beginnt mit: {api_key[:10]}...")

✓ OpenAI API Key geladen
  Key beginnt mit: sk-proj-Bk...


## Embedding-Modell laden

In [7]:
# Embedding-Modell für Query-Erstellung laden (gleich wie beim Indexieren)
model_name = "intfloat/multilingual-e5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()

print(f"Embedding-Modell geladen: {model_name}")

✓ Embedding-Modell geladen: intfloat/multilingual-e5-small


## ChromaDB Collection laden

In [8]:
# ChromaDB Client und Collection laden
chroma_client = chromadb.PersistentClient(path=chroma_collection_db)
collection = chroma_client.get_collection(name=chroma_collection)

print(f"ChromaDB Collection geladen: {chroma_collection}")
print(f"  Anzahl Dokumente: {collection.count()}")

✓ ChromaDB Collection geladen: geschichtsbuch_collection
  Anzahl Dokumente: 369


## Hilfsfunktionen definieren

In [9]:
def create_query_embedding(query_text):
    """
    Erstellt ein Embedding für eine Query-Frage
    """
    query_with_prefix = f"query: {query_text}"
    query_encoded = tokenizer(
        [query_with_prefix],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )
    
    with torch.no_grad():
        query_outputs = model(**query_encoded)
        attention_mask = query_encoded['attention_mask']
        token_embeddings = query_outputs.last_hidden_state
        
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        query_embedding = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        query_embedding = torch.nn.functional.normalize(query_embedding, p=2, dim=1)
        query_embedding = query_embedding.cpu().numpy()[0]
    
    return query_embedding


def search_similar_chunks(query_text, n_results=3):
    """
    Sucht ähnliche Chunks in ChromaDB basierend auf der Query
    """
    query_embedding = create_query_embedding(query_text)
    
    results = collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=n_results
    )
    
    return results


def format_context(results):
    """
    Formatiert die gefundenen Chunks als Kontext für das LLM
    """
    context_parts = []
    
    for i, (doc, metadata) in enumerate(zip(results['documents'][0], results['metadatas'][0])):
        pages = metadata.get('page_numbers', 'N/A')
        context_parts.append(f"[Quelle {i+1}, Seite(n): {pages}]\n{doc}")
    
    return "\n\n---\n\n".join(context_parts)


print("Hilfsfunktionen definiert")

✓ Hilfsfunktionen definiert


## RAG-Funktion mit GPT-4o

In [10]:
def answer_question(question, n_results=3, temperature=0.7):
    """
    Beantwortet eine Frage mit GPT-4o unter Verwendung von RAG
    
    Args:
        question: Die zu beantwortende Frage
        n_results: Anzahl der zu suchenden relevanten Chunks
        temperature: Kreativität der Antwort (0.0 - 1.0)
    
    Returns:
        dict mit 'answer', 'sources' und 'context'
    """
    print(f"Suche relevante Informationen für: '{question}'")
    
    # 1. Relevante Chunks finden
    results = search_similar_chunks(question, n_results=n_results)
    
    # 2. Kontext formatieren
    context = format_context(results)
    
    print(f"{len(results['documents'][0])} relevante Chunks gefunden")
    print("Generiere Antwort mit GPT-4o...")
    
    # 3. System-Prompt und User-Prompt erstellen
    system_prompt = """Du bist ein hilfreicher Assistent, der Fragen basierend auf bereitgestellten Quelltexten beantwortet.

Regeln:
- Beantworte die Frage nur basierend auf den bereitgestellten Quellen
- Zitiere die relevanten Quellen in deiner Antwort (z.B. [Quelle 1])
- Wenn die Antwort nicht in den Quellen zu finden ist, sage das ehrlich
- Gib präzise und sachliche Antworten
- Antworte auf Deutsch
- Beschränke dich auf 2 Sätze"""

    user_prompt = f"""Kontext aus den Quelltexten:

{context}

---

Frage: {question}

Bitte beantworte die Frage basierend auf dem obigen Kontext."""

    # 4. GPT-4o API aufrufen
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=temperature,
        max_tokens=1000
    )
    
    answer = response.choices[0].message.content
    
    print("Antwort generiert!\n")
    
    # 5. Ergebnis zurückgeben
    return {
        'answer': answer,
        'sources': results['metadatas'][0],
        'context': context,
        'distances': results['distances'][0]
    }


print("RAG-Funktion definiert")

✓ RAG-Funktion definiert


## Test: Frage stellen

In [11]:
# Beispiel-Frage
question = "Wer war Wilhelm Tell?"

# Frage beantworten
result = answer_question(question, n_results=3)

# Ausgabe formatieren
print("="*80)
print(f"FRAGE: {question}")
print("="*80)
print(f"\nANTWORT:\n{result['answer']}")
print("\n" + "="*80)
print("QUELLEN:")
print("="*80)
for i, source in enumerate(result['sources']):
    distance = result['distances'][i]
    pages = source.get('page_numbers', 'N/A')
    print(f"\n{i+1}. Seite(n): {pages} | Ähnlichkeit: {1-distance:.3f}")
print("\n" + "="*80)

🔍 Suche relevante Informationen für: 'Wer war Wilhelm Tell?'
✓ 3 relevante Chunks gefunden
💭 Generiere Antwort mit GPT-4o...
✓ Antwort generiert!

FRAGE: Wer war Wilhelm Tell?

ANTWORT:
Wilhelm Tell war ein trefflicher Bogenschütze aus Bürzeln, der bekannt wurde, weil er sich weigerte, einem aufgestellten österreichischen Hut die gebührende Ehre zu erweisen. Dies führte dazu, dass er vor den Vogt Geßler gebracht wurde, der ihm zur Strafe befahl, einen Apfel vom Kopf seines eigenen Kindes zu schießen. Tell gelang dies, nachdem er zu Gott gebetet und einen zweiten Pfeil in den Köcher gesteckt hatte. Als Geßler ihn fragte, was dieser zweite Pfeil zu bedeuten habe, antwortete Tell, dass dieser für Geßlers Herz bestimmt gewesen wäre, hätte der erste Pfeil sein Kind getroffen [Quelle 2, 3].

QUELLEN:

1. Seite(n): 59 | Ähnlichkeit: 0.621

2. Seite(n): 57, 58, 59 | Ähnlichkeit: 0.588

3. Seite(n): 57 | Ähnlichkeit: 0.582



## Interaktive Q&A Schleife

In [12]:
# Interaktive Fragen-Loop
# Hinweis: In Jupyter funktioniert input() - zum Beenden 'exit' eingeben

def interactive_qa():
    """
    Interaktive Q&A-Schleife
    """
    print("="*80)
    print("Q&A System mit GPT-4o")
    print("="*80)
    print("Stelle deine Fragen. Beenden mit 'exit', 'quit' oder 'q'\n")
    
    while True:
        question = input("Deine Frage: ").strip()
        
        if question.lower() in ['exit', 'quit', 'q', '']:
            print("\nAuf Wiedersehen!")
            break
        
        print()
        result = answer_question(question, n_results=3)
        
        print("="*80)
        print("ANTWORT:")
        print("="*80)
        print(result['answer'])
        print("\n" + "-"*80)
        print("Quellen:")
        for i, source in enumerate(result['sources']):
            pages = source.get('page_numbers', 'N/A')
            print(f"  [{i+1}] Seite(n): {pages}")
        print("="*80 + "\n")

# Starte die interaktive Schleife
# interactive_qa()  # Auskommentiert - manuell starten wenn gewünscht

print("Interaktive Q&A-Funktion definiert")
print("  Starten mit: interactive_qa()")

✓ Interaktive Q&A-Funktion definiert
  Starten mit: interactive_qa()


In [14]:
interactive_qa()

Q&A System mit GPT-4o
Stelle deine Fragen. Beenden mit 'exit', 'quit' oder 'q'


🔍 Suche relevante Informationen für: 'was ist der inhalt des buches'
✓ 3 relevante Chunks gefunden
💭 Generiere Antwort mit GPT-4o...
✓ Antwort generiert!

ANTWORT:
Der Inhalt des Buches scheint sich mit geschichtlichen und verfassungsgeschichtlichen Themen zu befassen. Es behandelt die Darstellung gesetzgeberischer Tätigkeiten, den Kampf gegen die Aristokratie, die Erwerbung der tessinischen Vogteien sowie verschiedene kultur- und verfassungsgeschichtliche Rückblicke. Das Buch ist so konzipiert, dass es den freien Vortrag des Lehrers unterstützt und den Schülern ermöglicht, die Inhalte zu Hause zu vertiefen. Es enthält ebenfalls Abschnitte über religiöse Verhältnisse, Erwerbsleben, Schulwesen, Kunst und Wissenschaft [Quelle 1]. 

Zusätzlich gibt es Berichte über historische Ereignisse und Figuren, etwa die Geschichte von Wilhelm Tell, der gezwungen wird, einen Apfel vom Kopf seines Sohnes zu schießen, und 